In [ ]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv("/content/online_retail_II.csv")

# Display the first few rows and get info
print(df.head())
print(df.info())
print(df.describe())

  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS      12.0   
1  489434    79323P                   PINK CHERRY LIGHTS      12.0   
2  489434    79323W                  WHITE CHERRY LIGHTS      12.0   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE       48.0   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX      24.0   

           InvoiceDate  Price  Customer ID         Country  
0  2009-12-01 07:45:00   6.95      13085.0  United Kingdom  
1  2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
2  2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
3  2009-12-01 07:45:00   2.10      13085.0  United Kingdom  
4  2009-12-01 07:45:00   1.25      13085.0  United Kingdom  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 814845 entries, 0 to 814844
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice     

In [ ]:
# Drop rows where CustomerID is missing
df.dropna(subset=['Customer ID'], inplace=True)

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [ ]:
# Keep only rows with positive quantity
df = df[df['Quantity'] > 0]

In [ ]:
df['TotalPrice'] = df['Quantity'] * df['Price']

In [ ]:
# Save the cleaned data to a CSV file
df.to_csv('cleaned_retail_data.csv', index=False)

In [ ]:
df_cleaned = pd.read_csv('cleaned_retail_data.csv')

In [ ]:
import pandas as pd
import sqlite3

# --- 1. Load and Clean Data ---
print("Step 1: Loading and cleaning data...")
try:
    df = pd.read_csv("/content/cleaned_retail_data.csv")
except FileNotFoundError:
    print("\nERROR: Make sure 'Online Retail.xlsx' is in the same directory as this script.")
    exit()

# Drop rows where CustomerID is missing
df.dropna(subset=['Customer ID'], inplace=True)
# Keep only rows with positive quantity (sales, not returns)
df = df[df['Quantity'] > 0]
# Drop any duplicate rows
df.drop_duplicates(inplace=True)

# --- 2. Ensure Correct Data Types (CRITICAL STEP) ---
print("Step 2: Converting data types...")
# Convert InvoiceDate to datetime objects
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
# Create TotalPrice column
df['TotalPrice'] = df['Quantity'] * df['Price']


# --- 3. Recreate the Database and Table ---
print("Step 3: Creating fresh database 'ecommerce.db'...")
db_name = 'ecommerce.db'
table_name = 'retail'
conn = sqlite3.connect(db_name)

# Use 'if_exists='replace'' to ensure we start with a clean slate
df.to_sql(table_name, conn, if_exists='replace', index=False)

print(f"Table '{table_name}' created successfully in '{db_name}'.")


# --- 4. Run the SQL Query Directly ---
print("\nStep 4: Running the SQL query...")
sql_query = """
SELECT
    strftime('%Y-%m', InvoiceDate) AS SalesMonth,
    SUM(TotalPrice) AS MonthlyRevenue
FROM
    retail
GROUP BY
    SalesMonth
ORDER BY
    SalesMonth;
"""
# Use pandas to execute the query and get the result
try:
    result_df = pd.read_sql_query(sql_query, conn)
    print("\n--- QUERY RESULT ---")
    if result_df.empty:
        print("The query ran but returned no data. The source data might be the issue.")
    else:
        # Print the results from the DataFrame
        print(result_df)

        result_df.to_csv('monthly_revenue.csv', index=False)
        print("\nSuccess! Query output saved to 'monthly_revenue.csv'.")

    print("--------------------")

except Exception as e:
    print(f"\nAn error occurred while querying the database: {e}")

finally:
    # Close the connection
    conn.close()

Step 1: Loading and cleaning data...
Step 2: Converting data types...
Step 3: Creating fresh database 'ecommerce.db'...
Table 'retail' created successfully in 'ecommerce.db'.

Step 4: Running the SQL query...

--- QUERY RESULT ---
   SalesMonth  MonthlyRevenue
0     2009-12      683504.010
1     2010-01      555802.672
2     2010-02      504558.956
3     2010-03      696978.471
4     2010-04      591982.002
5     2010-05      597833.380
6     2010-06      636371.130
7     2010-07      589736.170
8     2010-08      602224.600
9     2010-09      829013.951
10    2010-10     1033112.010
11    2010-11     1166460.022
12    2010-12      570422.730
13    2011-01      568101.310
14    2011-02      446084.920
15    2011-03      594081.760
16    2011-04      468374.331
17    2011-05      677355.150
18    2011-06      660046.050
19    2011-07      598962.901
20    2011-08       65576.690

Success! Query output saved to 'monthly_revenue.csv'.
--------------------


In [ ]:
# Convert the 'InvoiceDate' column to a datetime type
df_cleaned['InvoiceDate'] = pd.to_datetime(df_cleaned['InvoiceDate'])

# Use the cleaned dataframe loaded earlier
snapshot_date = df_cleaned['InvoiceDate'].max() + pd.Timedelta(days=1)

df_rfm = df_cleaned.groupby('Customer ID').agg({
    'InvoiceDate': lambda date: (snapshot_date - date.max()).days,
    'Invoice': 'nunique',
    'TotalPrice': 'sum'
})

# Rename columns
df_rfm.rename(columns={'InvoiceDate': 'Recency',
                       'Invoice': 'Frequency',
                       'TotalPrice': 'MonetaryValue'}, inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Log transform to handle skewed data
rfm_log = np.log1p(df_rfm)

# Scale the data
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_log)

In [ ]:
# Find optimal k
sse = {}
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(rfm_scaled)
    sse[k] = kmeans.inertia_
# Plot SSE to find the elbow (e.g., k=3 or 4)

In [ ]:
# Assuming k=4 from elbow method
kmeans = KMeans(n_clusters=4, random_state=42)
df_rfm['Cluster'] = kmeans.fit_predict(rfm_scaled)

# Save the results with customer segments
df_rfm.to_csv('customer_segments.csv')